<center>
<img src="https://supportvectors.ai/logo-poster-transparent.png" width="400px" style="opacity:0.7">
</center>

## Working with Qdrant

In [1]:
%run supportvectors-common.ipynb


<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



First make sure your qdrant docker is running using the below steps:

- **If you are on a Mac or Windows, start your Docker Desktop first.**
- Then follow the below steps
- On a Mac, you could also create a volume through the Docker Desktop GUI and then associate the named volume in the second step below

```bash
# Pull Qdrant image
docker pull qdrant/qdrant

# Run Qdrant (on linux)
docker run -p 6333:6333 -p 6334:6334 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    qdrant/qdrant

## if on Mac (replace <my_volume> with the volume you created in the GUI)
docker run -p 6333:6333 -p 6334:6334 \
    -v <my_volume>:/qdrant/storage \
    qdrant/qdrant
```


### Imports

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from qdrant_client.models import Filter, FieldCondition, MatchValue
from rich import print as rprint
import numpy as np


### Connect to Qdrant and create a collection

In [3]:

# Connect to Qdrant
client = QdrantClient("localhost", port=6333)

# Create collection
collection_name = "my_collection"
vector_size = 128

# Check if collection exists
try:
    client.get_collection(collection_name)
    print(f"Collection '{collection_name}' already exists. Skipping creation.")
except Exception:
    print(f"Collection '{collection_name}' does not exist. Creating now.")
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=vector_size,
            distance=Distance.COSINE
        )
    )

Collection 'my_collection' does not exist. Creating now.


### Basic operations

#### Insert vectors

In [4]:
# Prepare data
vector1 = np.random.rand(128).astype(np.float32)
vector2 = np.random.rand(128).astype(np.float32)
points = [
    PointStruct(
        id=1,
        vector=vector1,
        payload={"text": "sample text", "category": "news"}
    ),
    PointStruct(
        id=2,
        vector=vector2,
        payload={"text": "another text", "category": "sports"}
    )
]

# Insert points
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

#### Search vectors

In [5]:
# Basic search
search_result = client.search(
    collection_name=collection_name,
    query_vector=vector1,
    limit=10
)


In [6]:
rprint(search_result)

[
    ScoredPoint(
        id=1,
        version=0,
        score=0.9999999,
        payload={'text': 'sample text', 'category': 'news'},
        vector=None,
        shard_key=None,
        order_value=None
    ),
    ScoredPoint(
        id=2,
        version=0,
        score=0.73362184,
        payload={'text': 'another text', 'category': 'sports'},
        vector=None,
        shard_key=None,
        order_value=None
    )
]

In [7]:

# Search with filtering

search_result = client.search(
    collection_name=collection_name,
    query_vector=vector1,
    query_filter=Filter(
        must=[
            FieldCondition(
                key="category",
                match=MatchValue(value="sports")
            )
        ]
    ),
    limit=10
)

In [8]:
rprint(search_result)

[
    ScoredPoint(
        id=2,
        version=0,
        score=0.73362184,
        payload={'text': 'another text', 'category': 'sports'},
        vector=None,
        shard_key=None,
        order_value=None
    )
]